In [23]:
%matplotlib inline

import os as os
import collections as col
import itertools as itt
import pickle as pck
import time as ti

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import numpy as np
import numpy.random as rng
import scipy.stats as stats
import pandas as pd
import seaborn as sns

# What does this do?
# Plot a heatmap of state transitions
# observed in HSPs between replicate
# comparisons

date = '20180409'

run_plot_hsp_trans_freq = True

save_figures = True

sns.set(style='white',
        font_scale=1.5,
        rc={'font.family': ['sans-serif'],
            'font.sans-serif': ['DejaVu Sans']})

fhgfs_base = '/TL/deep/fhgfs/projects/pebert/thesis/projects/statediff'
cache_dir = os.path.join(fhgfs_base, 'caching/notebooks')

hsp_files_folder = os.path.join(fhgfs_base, 'solidstate/deep')

base_out = '/TL/deep-external01/nobackup/pebert/cloudshare/mpiinf/phd/chapter_projects/statediff'
fig_supp = os.path.join(base_out, 'figures', 'pub', 'supp')
fig_main = os.path.join(base_out, 'figures', 'pub', 'main')
fig_collect = os.path.join(base_out, 'figures', 'pub', 'collection')
                   

def create_heatmap(data, title, xlab, ylab):
    """
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(data, vmin=0, vmax=1, cmap='YlOrRd', square=True,
                annot=False, cbar=True, linewidths=1, linecolor='lightgrey',
                xticklabels=True, yticklabels=True, ax=ax)
    ax.set_xlabel(xlab, fontsize=16)
    ax.set_ylabel(ylab, fontsize=16)
    tt = ax.set_title(title)
    tt.set_position([0.5, 1.03])
    return fig, []
    
            
def plot_hsp_trans_freq():
    
    base_path = os.path.join(fhgfs_base, 'solidstate', 'deep')
    for root, dirs, datafiles in os.walk(base_path):
        if datafiles:
            for df in datafiles:
                if df.startswith('hsp_trans') and df.endswith('.tsv'):
                    _, _, ylab, _, xlab = df.split('.')[0].split('_')
                    fpath = os.path.join(root, df)
                    data = pd.read_csv(fpath, sep='\t', index_col=0)
                    data = data.loc[data.index[:-1], data.columns[:-1]].copy()
                    for i in range(data.shape[0]):
                        data.iloc[i, i] = np.nan
                    row_totals = data.sum(axis=1)
                    for i in range(data.shape[0]):
                        data.iloc[i, :] = data.iloc[i, :] / row_totals[i]
                    num_hsp, _ = data.index.name.split('_')
                    fig_title = 'State transitions in {} HSPs'.format(num_hsp)
                    fig, exart = create_heatmap(data, fig_title,
                                                'To {}'.format(xlab),
                                                'From {}'.format(ylab))
                    if save_figures:
                        outname = '{}_fig_X_hsp_transfreq_{}_{}_{}_vs_{}'.format(date, 'cmm18', 'emission', ylab, xlab)
                        out_svg = os.path.join(fig_collect, outname + '.svg')
                        fig.savefig(out_svg, bbox_inches='tight', extra_artists=exart)
                        out_pdf = os.path.join(fig_collect, outname + '.pdf')
                        fig.savefig(out_pdf, bbox_inches='tight', extra_artists=exart)
                        out_png = os.path.join(fig_collect, outname + '.png')
                        fig.savefig(out_png, bbox_inches='tight', extra_artists=exart, dpi=300)
                    plt.close(fig)
    return 0
     
    
if run_plot_hsp_trans_freq:
    plot_hsp_trans_freq()
